In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager
from random_user_agent.user_agent import UserAgent 
from random_user_agent.params import SoftwareName, OperatingSystem 
from time import sleep
import logging
import traceback
from self import self

class Request:
    logger=logging.getLogger('vivino.project.requests')
    selenium_retries=0
    
    def __init__(self,url):
        self.url = url
        
    def get_selenium_res(self,class_name):
        try:
            software_names=[SoftwareName.CHROME.value]
            operating_systems=[OperatingSystem.WINDOWS.value,
                                  OperatingSystem.LINUX.value]
            user_agent_rotator=UserAgent(software_names=software_names,
                                            operating_systems=operating_systems,
                                            limit=100)
            user_agent=user_agent_rotator.get_random_user_agent()
                
            chrome_options=Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--window-size=1420,1080")
            chrome_options.add_argument("--disable-gpu")
            chrome_options.add_argument(f'user-agent={user_agent}')

            #PROXY="http://73.192.128.238:25"
                
            #if not IS_GAE_PRODUCTION_ENV:
            #chrome_options.binary_location='/Users/lilly/Documents/chromedriver'
                
            #prox=Proxy()
            #prox.proxy_type=ProxyType.MANUAL
            #prox.autodetect=False
            #capabilities=webdriver.DesiredCapabilities.CHROME
            #prox.http_proxy=PROXY
            #prox.ssl_proxy=PROXY
            #prox.add_to_capabilities(capabilities)
                
            #browser =webdriver.Chrome('/Users/lilly/Desktop/chromedriver',options=chrome_options,
            #                      desired_capabilities=capabilities)
                
            browser = webdriver.Chrome('/Users/lilly/Desktop/chromedriver',options=chrome_options)
                                           
            # when testing proxies 
            #browser.get('http://lumtest.com/myip.json')
                                           
            browser.get(self.url)
                
            time_to_wait=90
            try:
                WebDriverWait(browser,time_to_wait).until(
                EC.presence_of_element_located((By.CLASS_NAME, class_name)))
            finally:
                browser.maximize_window()
                page_html = browser.page_source
                browser.close()
                return page_html
        except (TimeoutException,WebDriverException):
            self.logger.error(traceback.format_exc())
            sleep(6)
            self.selenium_retries += 1
            self.logger.info('Selenium retry #: '+ str(self.selenium_retries))
            return self.get_selenium_res(class_name)
                
                

In [673]:
import pandas as pd

#Henter csv fil med vin data
vivino=pd.read_csv('vivino_v7.csv',index_col=[0])

#Drop duplicates by link
vivino=vivino.drop_duplicates(['link']).reset_index(drop=True)

#remove year from 'name':
a=0
for i in vivino['name']:
    vivino['name'][a]=re.sub(r"\d", "", i)
    a+=1

#drop duplicates by name 
vivino=vivino.drop_duplicates(['name']).reset_index(drop=True)

vivino.to_csv('final_vivino.csv')

/Users/lilly/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [919]:
from bs4 import BeautifulSoup
import re
import numpy as np


B=[]

for a in vivino['link'][544:580]:
    A=[]
    soup = BeautifulSoup(Request(a).get_selenium_res('indicatorBar__progress--3aXLX'), 'lxml')
    
    #Indhenter gennemsnitsprisen
    p=soup.find_all('span', {'class': 'purchaseAvailabilityPPC__amount--2_4GT'})

    if not p:
        g=soup.find_all('span', {'class': 'purchaseAvailability__currentPrice--3mO4u'})
        if not g:
            A.append(np.nan)
        else:
            A.append(soup.find_all('span', {'class': 'purchaseAvailability__currentPrice--3mO4u'})[0].text)
    else:
        A.append(soup.find_all('span', {'class': 'purchaseAvailabilityPPC__amount--2_4GT'})[0].text)
    
    # Indhenter smagsnoter
    bar=soup.find_all('span', {'class': 'indicatorBar__progress--3aXLX'})
    if not bar:
        for i in range(0,4):
            A.append(np.nan)
    else:
        for i in range(0,4):
            A.append(re.sub(r'[^\d\.]','',bar[i]['style'][18:]))
        
    #Indhenter alcohol%
    script = str(soup.find_all("script"))
    n=[]
    for match in re.finditer(r'("alcohol":)',script):
        n.append(match.span(0))
    span=len(n)

    if span==0 or span==1:
        A.append(np.nan)
    else:
        m=script[n[span-1][span-1]:]
        if bool(re.match(r'(minimum)', re.search(r'^(.+?),', m)[1]))==True:
            A.append(np.nan)
        else:
            A.append(re.sub(r'\}','',re.search(r'^(.+?),', m)[1]))

    #Indhenter sukker pr. gram
    if re.search(r'(residual_sugar_grams_per_liter)',script)==None:
        A.append(np.nan)
    else:
        n=re.search(r'(residual_sugar_grams_per_liter)',script).span(0)[1]
        m=script[ n+2:]
        A.append(re.search(r'^(.+?),', m)[1])

    # Indhenter syre pr.liter
    if re.search(r'(acidity_grams_per_liter)',script)==None:
        A.append(np.nan)
    else:
        n=re.search(r'(acidity_grams_per_liter)',script).span(0)[1]
        m=script[n+2:]
        A.append(re.search(r'^(.+?),', m)[1])

    # Winery
    A.append(re.sub("(\n)", "", soup.find_all('a', {'class': 'winery'})[0].text))
    
    B.append(A)

IndexError: list index out of range

In [915]:
soup

<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">Your IP address (77.241.128.37) has been temporarily blocked for exceeding bulk request limits. If you believe this was done in error or you have legitimate needs to access our pages and data above and beyond these limits please contact admin@vivino.com with the subject 'Requests Blocked' and we'll try and resolve the issue.</pre></body></html>

In [920]:
len(B)

0

In [894]:
#for a in range(120,200):
#    print(vivino['link'][a])

In [916]:
df=pd.DataFrame(B, columns=['price','light-bold','smooth-tannic','dry-sweet','soft-acidic','alcohol_pct','residual_sugar_g/l','acidity_g/l','winery'])
df

,price,light-bold,smooth-tannic,dry-sweet,soft-acidic,alcohol_pct,residual_sugar_g/l,acidity_g/l,winery
0,610.78 kr.,47.779,58.8047,9.36773,63.6244,NaN,NaN,NaN,Mastrojanni
1,330 kr.,84.1444,18.651,49.3138,27.1657,NaN,NaN,NaN,Tenute Fiorebelli
2,578 kr.,63.3535,77.216,8.39561,79.7502,NaN,NaN,NaN,Parusso
3,899 kr.,84.0701,48.121,29.1226,31.0959,15.5,NaN,NaN,Passopisciaro
4,299 kr.,64.6116,40.7656,9.59083,42.6548,NaN,NaN,NaN,Morami
5,857.55 kr.,50.8072,49.1518,4.22471,49.1268,13.5,NaN,NaN,Querciabella
6,597.93 kr.,61.1389,80.8667,1.11667,79.8222,NaN,NaN,NaN,Moscone Brothers
7,766.08 kr.,61.1585,75.2203,0,81.3534,14,NaN,NaN,Prunotto
8,943.84 kr.,70.4375,62.625,9.1875,62.25,NaN,NaN,NaN,Bueno
9,495 kr.,85,20.205,49.6773,25.8073,NaN,NaN,NaN,La Giaretta


In [917]:
merged=pd.concat([df2,df]).reset_index(drop=True)
merged

,price,light-bold,smooth-tannic,dry-sweet,soft-acidic,alcohol_pct,residual_sugar_g/l,acidity_g/l,winery
0,2050.86 kr.,72.213,40.0278,18,20.8148,NaN,NaN,NaN,Marion
1,11500 kr.,61.6076,80.2688,9.65777,82.9995,NaN,NaN,NaN,Giacomo Conterno
2,1100 kr.,85,27.6326,46.497,27.5195,NaN,NaN,NaN,Tedeschi
3,1251.36 kr.,61.3094,77.8291,7.14819,82.4921,NaN,NaN,NaN,Vietti
4,825 kr.,85,19.9018,47.9276,20.3641,NaN,NaN,NaN,Ernesto Ruffo
...,...,...,...,...,...,...,...,...,...
539,261.84 kr.,55.6424,34.8159,5.4467,37.5736,NaN,NaN,NaN,Fattoria la Parrina
540,409 kr.,63.5355,78.2128,4.91967,85,NaN,NaN,NaN,Abbona
541,536.90 kr.,60.8409,79.6588,12.255,81.6581,14,NaN,NaN,Renato Ratti
542,666.65 kr.,84.2927,28.0038,46.3219,25.1144,NaN,NaN,NaN,Cesari


In [918]:
df2=merged
df2.to_csv('scraping.csv')
df2

,price,light-bold,smooth-tannic,dry-sweet,soft-acidic,alcohol_pct,residual_sugar_g/l,acidity_g/l,winery
0,2050.86 kr.,72.213,40.0278,18,20.8148,NaN,NaN,NaN,Marion
1,11500 kr.,61.6076,80.2688,9.65777,82.9995,NaN,NaN,NaN,Giacomo Conterno
2,1100 kr.,85,27.6326,46.497,27.5195,NaN,NaN,NaN,Tedeschi
3,1251.36 kr.,61.3094,77.8291,7.14819,82.4921,NaN,NaN,NaN,Vietti
4,825 kr.,85,19.9018,47.9276,20.3641,NaN,NaN,NaN,Ernesto Ruffo
...,...,...,...,...,...,...,...,...,...
539,261.84 kr.,55.6424,34.8159,5.4467,37.5736,NaN,NaN,NaN,Fattoria la Parrina
540,409 kr.,63.5355,78.2128,4.91967,85,NaN,NaN,NaN,Abbona
541,536.90 kr.,60.8409,79.6588,12.255,81.6581,14,NaN,NaN,Renato Ratti
542,666.65 kr.,84.2927,28.0038,46.3219,25.1144,NaN,NaN,NaN,Cesari
